In [1]:
import intake
import satstac
import yaml

import intake_stac
intake_stac.__version__

'0.3.0.post58'

# Load an Intake Catalog

In [2]:
url = 'https://planetarycomputer.microsoft.com/api/stac/v1'
#url = 'https://raw.githubusercontent.com/intake/intake-stac/main/intake_stac/tests/data/1.0.0/catalog/catalog.json'
stac_cat = intake.open_stac_catalog(url)
stac_cat

microsoft-pc:
  args:
    stac_obj: https://planetarycomputer.microsoft.com/api/stac/v1
  description: ''
  driver: intake_stac.catalog.StacCatalog
  metadata:
    conformsTo:
    - https://api.stacspec.org/v1.0.0-beta.2/core
    - https://api.stacspec.org/v1.0.0-beta.2/item-search
    - https://api.stacspec.org/v1.0.0-beta.2/item-search#query
    - http://www.opengis.net/spec/ogcapi-features-1/1.0/conf/core
    - http://www.opengis.net/spec/ogcapi-features-1/1.0/conf/oas30
    - http://www.opengis.net/spec/ogcapi-features-1/1.0/conf/geojson
    description: Searchable spatiotemporal metadata describing Earth science datasets
      hosted by the Microsoft Planetary Computer
    id: microsoft-pc
    stac_extensions: []
    stac_version: 1.0.0
    title: Microsoft Planetary Computer STAC API
    type: Catalog


# Serialize as a .YAML file

In [3]:
stac_cat.walk(depth=0)

{'mobi': name: mobi
 container: catalog
 plugin: ['stac_catalog']
 driver: ['stac_catalog']
 description: The [Map of Biodiversity Importance](https://www.natureserve.org/conservation-tools/projects/map-biodiversity-importance) (MoBI) consists of a series of raster maps that combine habitat information for 2,216 imperiled species occurring in the conterminous United States, using weightings based on range size and degree of protection to identify areas of high importance for biodiversity conservation. Species included in the project are those which, as of September 2018, had a global conservation status of G1 (critical imperiled) or G2 (imperiled) or which are listed as threatened or endangered at the full species level under the United States Endangered Species Act. Taxonomic groups included in the project are vertebrates (birds, mammals, amphibians, reptiles, turtles, crocodilians, and freshwater and anadromous fishes), vascular plants, selected aquatic invertebrates (freshwater muss

In [4]:
stac_cat.walk(depth=2) # Works! type dict
#stac_cat.walk(depth=10).serialize() # Attribute Error, no attribute `serialize`

#print(stac_cat.yaml()) # Works! type str
#stac_cat.yaml(depth=1) # Type Error, unexpected keyword `depth`, this is our target api

#print(stac_cat.test.yaml()) # type st
#print(stac_cat.test) #type StacCatalog

{'mobi': name: mobi
 container: catalog
 plugin: ['stac_catalog']
 driver: ['stac_catalog']
 description: The [Map of Biodiversity Importance](https://www.natureserve.org/conservation-tools/projects/map-biodiversity-importance) (MoBI) consists of a series of raster maps that combine habitat information for 2,216 imperiled species occurring in the conterminous United States, using weightings based on range size and degree of protection to identify areas of high importance for biodiversity conservation. Species included in the project are those which, as of September 2018, had a global conservation status of G1 (critical imperiled) or G2 (imperiled) or which are listed as threatened or endangered at the full species level under the United States Endangered Species Act. Taxonomic groups included in the project are vertebrates (birds, mammals, amphibians, reptiles, turtles, crocodilians, and freshwater and anadromous fishes), vascular plants, selected aquatic invertebrates (freshwater muss

In [5]:
#yaml.dump(stac_cat.walk(depth=2))
#print(yaml.dump(stac_cat)) # type str
print(yaml.dump(stac_cat.walk(depth=0))) # type str

#Missing source: and shouldn't have !!python/object:intake.catalog.local.LocalCatalogEntry to work

3dep-seamless: !!python/object:intake.catalog.local.LocalCatalogEntry
  args: []
  cls: intake.catalog.local.LocalCatalogEntry
  kwargs:
    name: 3dep-seamless
    description: The [USGS 3D Elevation Program (3DEP) Datasets](https://www.usgs.gov/core-science-systems/ngp/3dep)
      from [The National Map](https://www.usgs.gov/core-science-systems/national-geospatial-program/national-map)
      are the primary elevation data product produced and distributed by the USGS.
      The 3DEP program provides a variety of resolution raster elevation data of the
      conterminous United States, Alaska, Hawaii, and the island territories. Some
      of the data sets such as the 1/3rd arc-second and 1 arc-second data set are
      derived from diverse source data sets that are processed to a specification
      with a consistent resolution, coordinate system, elevation units, and horizontal
      and vertical datums.\n\nThe seamless DEM layers under the 3DEP program are a
      multi-resolution 

In [16]:
with open('test.yaml', 'w') as f:
    f.write(yaml.dump(stac_cat.walk(depth=2)))

Code from: https://github.com/intake/intake/blob/471488801bfa338dbfd91f9851b27580f4f3c3bb/intake/source/base.py#L242-L265showing how intake turns its catalogues into a dictionary with very specific keys before into a yaml.

```
def _yaml(self):
    import inspect
    kwargs = self._captured_init_kwargs.copy()
    meta = kwargs.pop('metadata', self.metadata) or {}
    kwargs.update(dict(zip(inspect.signature(self.__init__).parameters,
                       self._captured_init_args)))
    data = {
        'sources':
            {self.name: {
                'driver': self.classname,
                'description': self.description or "",
                'metadata': meta,
                'args': kwargs
            }}}
    return data

def yaml(self):
    """Return YAML representation of this data-source
    The output may be roughly appropriate for inclusion in a YAML
    catalog. This is a best-effort implementation
    """
    data = self._yaml()
    return yaml.dump(data, default_flow_style=False)
```

In [11]:
#`a` is a dictionary of `LocalCatalogEntry`s. I can pull out some attributes, but not others. Unsure why.

a = stac_cat.walk(depth=2)
#print(a)
#print(type(a))

#print(a['mobi'])
#print(type(a['mobi']))

print(a['mobi'].name)
#print(type(a['mobi'].name))

print(a['mobi'].driver) #fails

mobi


AttributeError: 'LocalCatalogEntry' object has no attribute 'driver'

In [12]:
#I'm trying to recreate the data dictionary from intake.yaml() without the references to self, but since driver can't be accessed it is stuck.

data = {
    'sources':
        {a['mobi'].name: {
            'driver': a['mobi'].driver,
            'description': a['mobi'].description or "",
            'metadata': a['mobi'].metadata,
            'args': a['mobi'].args
            }}}

AttributeError: 'LocalCatalogEntry' object has no attribute 'driver'

# Full Circle: Load Intake Catalogue from .YAML

In [13]:
catalog = intake.open_catalog('works.yaml') # Constructor Error

In [17]:
catalog2 = intake.open_catalog('test.yaml') # Constructor Error

ConstructorError: could not determine a constructor for the tag 'tag:yaml.org,2002:python/object:intake.catalog.local.LocalCatalogEntry'
  in "<unicode string>", line 1, column 16:
    3dep-seamless: !!python/object:intake.catalog.l ... 
                   ^